In [ ]:
import struct
import numpy as np
import os
import cv2

def decode_idx3_ubyte(idx3_ubyte_file):
    with open(idx3_ubyte_file, 'rb') as f:
        print('解析文件：', idx3_ubyte_file)
        fb_data = f.read()

    offset = 0
    fmt_header = '>iiii'    # 以大端法读取4个 unsinged int32
    magic_number, num_images, num_rows, num_cols = struct.unpack_from(fmt_header, fb_data, offset)
    print('魔数：{}，图片数：{}'.format(magic_number, num_images))
    offset += struct.calcsize(fmt_header)  # 索引跟随
    fmt_image = '>' + str(num_rows * num_cols) + 'B'

    images = np.empty((num_images, num_rows, num_cols))
    for i in range(num_images):
        im = struct.unpack_from(fmt_image, fb_data, offset)
        images[i] = np.array(im).reshape((num_rows, num_cols))
        offset += struct.calcsize(fmt_image)
    return images

def decode_idx1_ubyte(idx1_ubyte_file):
    with open(idx1_ubyte_file, 'rb') as f:
        print('解析文件：', idx1_ubyte_file)
        fb_data = f.read()

    offset = 0
    fmt_header = '>ii'  # 以大端法读取两个 unsinged int32
    magic_number, label_num = struct.unpack_from(fmt_header, fb_data, offset)
    print('魔数：{}，标签数：{}'.format(magic_number, label_num))
    offset += struct.calcsize(fmt_header)
    labels = []

    fmt_label = '>B'    # 每次读取一个 byte
    for i in range(label_num):
        labels.append(struct.unpack_from(fmt_label, fb_data, offset)[0])
        offset += struct.calcsize(fmt_label)
    return labels

def check_folder(folder):
    """检查文件文件夹是否存在，不存在则创建"""
    if not os.path.exists(folder):
        os.mkdir(folder)
        print(folder)
    else:
        if not os.path.isdir(folder):
            os.mkdir(folder)
            print(folder)


def export_img(exp_dir, img_ubyte, lable_ubyte):
    """
    生成数据集
    """
    check_folder(exp_dir)
    images = decode_idx3_ubyte(img_ubyte)
    labels = decode_idx1_ubyte(lable_ubyte)

    nums = len(labels)
    for i in range(nums):
        img_dir = os.path.join(exp_dir, str(labels[i]))
        check_folder(img_dir)
        img_file = os.path.join(img_dir, str(i)+'.png')
        imarr = images[i]
        cv2.imwrite(img_file, imarr)


def parser_mnist_data(data_dir):

    train_dir = os.path.join(data_dir, 'train')
    train_img_ubyte = os.path.join(data_dir, 'train-images.idx3-ubyte')
    train_label_ubyte = os.path.join(data_dir, 'train-labels.idx1-ubyte')
    export_img(train_dir, train_img_ubyte, train_label_ubyte)

    test_dir = os.path.join(data_dir, 'test')
    test_img_ubyte = os.path.join(data_dir, 't10k-images.idx3-ubyte')
    test_label_ubyte = os.path.join(data_dir, 't10k-labels.idx1-ubyte')
    export_img(test_dir, test_img_ubyte, test_label_ubyte)


# if __name__ == '__main__':
#     data_dir = 'G:/MNIST_data/'
#     parser_mnist_data(data_dir)


In [ ]:
#将原始图片转换成需要的大小，并将其保存位tfrecords格式
#========================================================================================
import os  
import tensorflow as tf  
from PIL import Image  
import random
  
# #原始图片的存储位置
# orig_picture = 'G:/MNIST_data/train/'

# #生成图片的存储位置
# gen_picture = 'G:/MNIST_data/'#E:/Re_train/image_data/inputdata/'

 
#需要的识别类型
classes = ['0','1','2','3','4','5','6','7','8','9']
# for index, name in enumerate(classes):
#         print(index,'--',name)

def create_txt(img_path):
    paths = []
    f = open(img_path+'/img.txt', 'w')
    for file_name in os.listdir(img_path):
        if os.path.isdir(img_path+file_name):
            for img_name in os.listdir(img_path+file_name):
#                 path = os.path.join(img_path, file_name, img_name)
                path = img_path+'/'+file_name+'/'+img_name+' '+file_name
                paths.append(path)
    random.shuffle(paths)
    for i in paths:
        f.write(i+'\n')
    f.close()
    
# create_txt('G:/MNIST_data/train/')
def create_record1(tf_path):  
    writer = tf.python_io.TFRecordWriter(tf_path)
    num_m = 0
    f = open('G:/MNIST_data/train/img.txt', 'r')
    contents = f.readlines()
    f.close()
    for content in contents:
#         print(content)
        image = content.split()
#         print(image)
        for index, name in enumerate(classes):
#             class_path = img_path + name+"/"  #每一个分类的图片路径"G:/MNIST_data/train/0/
#             print(image[1],'--',name)
            if image[1] == name:
                print('读取标签',index,'--',name,image[0])
#             for img_name in os.listdir(class_path):  #遍历路径下的文件
#                 imgs_path = class_path + img_name  #图片完整路径如"G:/MNIST_data/train/0/1.png"
#             print('图片';imgs_path)
#             print(index,'--',class_path)
                img = Image.open(image[0])  # 
#             img = cv2.imread(imgs_path, cv2.IMREAD_GRAYSCALE)
#             img = img.resize((28, 28))    #设置需要转换的图片大小
                img_raw = img.tobytes()      #将图片转化 img.tostring()
#             img_raw = img.tostring()
#             print(index,img_raw)
                example = tf.train.Example(  
                   features=tf.train.Features(feature={  
                        "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[index])),  
                        'image_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw]))  
                   }))  
                writer.write(example.SerializeToString())
                num_m +=1
    writer.close()
    print('the numbers of picture:', num_m)
    print(tf_path)
# create_record1('G:/MNIST_data/train/train.tfrecords')
   
# 制作TFRecords数据
def create_record(tf_path, img_path):  
    writer = tf.python_io.TFRecordWriter(tf_path)
    num_m = 0
    for index, name in enumerate(classes):
        class_path = img_path + name+"/"  #每一个分类的图片路径"G:/MNIST_data/train/0/
        print('读取',index,'--',name,class_path)
        for img_name in os.listdir(class_path):  #遍历路径下的文件
            imgs_path = class_path + img_name  #图片完整路径如"G:/MNIST_data/train/0/1.png"
#             print('图片';imgs_path)
#             print(index,'--',class_path)
            img = Image.open(imgs_path)  # 
#             img = cv2.imread(imgs_path, cv2.IMREAD_GRAYSCALE)
#             img = img.resize((28, 28))    #设置需要转换的图片大小
            img_raw = img.tobytes()      #将图片转化 img.tostring()
#             img_raw = img.tostring()
#             print(index,img_raw)
            example = tf.train.Example(  
               features=tf.train.Features(feature={  
                    "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[index])),  
                    'image_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw]))  
               }))  
            writer.write(example.SerializeToString())
            num_m +=1
    writer.close()
    print('the numbers of picture:', num_m)
    print(tf_path)

tr_tf_path, tr_img_path = 'G:/MNIST_data/train/mnist_train.tfrecords', 'G:/MNIST_data/train/'
te_tf_path, te_img_path = 'G:/MNIST_data/test/mnist_test.tfrecords', 'G:/MNIST_data/test/'
# create_record(tr_tf_path, tr_img_path)
# create_record(te_tf_path, te_img_path)



In [ ]:
#读取frecords并还原为图片
#=======================================================================================
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

def read_and_decode(filename):  
    # 创建文件队列,不限读取的数量  
    filename_queue = tf.train.string_input_producer([filename])  
    # create a reader from file queue  
    reader = tf.TFRecordReader()  
    # reader从文件队列中读入一个序列化的样本  
    _, serialized_example = reader.read(filename_queue)  
    # get feature from serialized example  
    # 解析符号化的样本  
    features = tf.parse_single_example(  #按原保存的格式解析
        serialized_example,  
        features={  
            'label': tf.FixedLenFeature([], tf.int64),  
            'image_raw': tf.FixedLenFeature([], tf.string)  
        })  
    label = features['label']  
    img = features['image_raw']  
    de_img = tf.decode_raw(img, tf.uint8) #图像解析为对应的像素数组
    
    image = tf.reshape(de_img, [28, 28]) #   
    labels = tf.cast(label, tf.int32)  
    return image, labels 

def get_tf(batch_size, Train=True):
    if Train:
        tf_path = tr_tf_path
    else:
        tf_path = te_tf_path
    img, label = read_and_decode(tf_path)
    img_batch, label_batch = tf.train.shuffle_batch([img,label],
                                                   batch_size=batch_size,
                                                   num_threads=2,
                                                   capacity=1000,
                                                   min_after_dequeue=700)
    return img_batch, label_batch

#=======================================================================================
if __name__ == '__main__':
    #样本总数
    num_samples = 60000
    num_samples_ = 10000
    #文件
#     filename = "G:/MNIST_data/test/mnist_train.tfrecords"
    batch = read_and_decode(tr_tf_path)
#     batch = get_tf(batch_size=100, Train=False)
#     init_op = tf.global_variables_initializer() 
      
    with tf.Session() as sess: #开始一个会话
#         sess.run(tf.global_variables_initializer())
#         sess.run(tf.local_variables_initializer())
#         sess.run(init_op)    
        coord=tf.train.Coordinator()    
        threads= tf.train.start_queue_runners(coord=coord)  
        
        
        for i in range(num_samples_):    
            example, lab = sess.run(batch)#在会话中取出image和label
#             print(example.shape, lab.shape)
            img=Image.fromarray(example,'L')#这里Image是之前提到的
    
#             for index, name in enumerate(classes):
#                 if lab == index:
#                     check_folder('G:/MNIST_data/'+name)
#                     img.save('G:/MNIST_data/'+name+'/'+str(i)+'.png')
                    
#             for j in np.arange(100):

#                     # np.arange()函数返回一个有终点和起点的固定步长的排列
#                 img=Image.fromarray(example[j],'L')
#                 print('label: %d' % lab[j])

#                 plt.imshow(img)

#                 title = classes[int(lab[j])]

#                 plt.title(title)
                    
#                 plt.axis('off')
                    
#                 plt.show()     
        
            
            if i == 6000:
                print(lab)
                plt.imshow(img)
#             if lab == 1:
#                 check_folder('G:/MNIST_data/'+str(lab))
#                 img.save('G:/MNIST_data/'+str(lab)+'/'+str(i)+'.png')
                    
#             if lab==2:
#                 check_folder('G:/MNIST_data/'+str(lab))
#                 img.save('G:/MNIST_data/'+str(lab)+'/'+str(i)+'.png')#存下图片;注意cwd后边加上‘/’ 
#             if lab==3:
#                 check_folder('G:/MNIST_data/'+str(lab))
#                 img.save('G:/MNIST_data/'+str(lab)+'/'+str(i)+'.png')
#             if lab==4:
#                 check_folder('G:/MNIST_data/'+str(lab))
#                 img.save('G:/MNIST_data/'+str(lab)+'/'+str(i)+'.png')            
        coord.request_stop()
        coord.join(threads)

In [ ]:

'''
PreWork.py

功能：实现对指定大小的生成图片进行sample与label分类制作

获得神经网络输入的get_files文件，同时为了方便网络的训练，输入数据进行batch处理。

'''


import os

import numpy as np

from PIL import Image

import tensorflow as tf

import matplotlib.pyplot as plt

from numpy import *

 

angry = []

label_angry = []

disgusted = []

label_disgusted = []

fearful = []

label_fearful = []

happy = []

label_happy = []

sadness = []

label_sadness = []

surprised = []

label_surprised = []

 

 

def get_file(file_dir):

    # step1：获取路径下所有的图片路径名，存放到

    # 对应的列表中，同时贴上标签，存放到label列表中。

    for file in os.listdir(file_dir + '/0'):

        angry.append(file_dir + '/0' + '/' + file)

        label_angry.append(0)

    for file in os.listdir(file_dir + '/1'):

        disgusted.append(file_dir + '/1' + '/' + file)

        label_disgusted.append(1)

    for file in os.listdir(file_dir + '/2'):

        fearful.append(file_dir + '/2' + '/' + file)

        label_fearful.append(2)

    for file in os.listdir(file_dir + '/3'):

        happy.append(file_dir + '/3' + '/' + file)

        label_happy.append(3)

    for file in os.listdir(file_dir + '/4'):

        sadness.append(file_dir + '/4' + '/' + file)

        label_sadness.append(4)

    for file in os.listdir(file_dir + '/5'):

        surprised.append(file_dir + '/5' + '/' + file)

        label_surprised.append(5)

 

    # 打印出提取图片的情况，检测是否正确提取

    print("There are %d 0\nThere are %d 1\nThere are %d 2\n" %(len(angry), len(disgusted), len(fearful)),end="")

    print("There are %d 3\nThere are %d 4\nThere are %d 5\n" %(len(happy),len(sadness),len(surprised)))

 

    # step2：对生成的图片路径和标签List做打乱处理把所有的合起来组成一个list（img和lab）

    # 合并数据numpy.hstack(tup)

    # tup可以是python中的元组（tuple）、列表（list），或者numpy中数组（array），函数作用是将tup在水平方向上（按列顺序）合并

    image_list = np.hstack((angry, disgusted, fearful, happy, sadness, surprised))

    label_list = np.hstack((label_angry, label_disgusted, label_fearful, label_happy, label_sadness, label_surprised))

    # 利用shuffle，转置、随机打乱

    temp = np.array([image_list, label_list])   # 转换成2维矩阵 [2, num]

    temp = temp.transpose()     # 转置  [num, 2]

    # numpy.transpose(a, axes=None) 作用：将输入的array转置，并返回转置后的array

    np.random.shuffle(temp)     # 按行随机打乱顺序函数 np.random.permutation(temp)
#     print(temp.shape)
 

    # 将所有的img和lab转换成list

    all_image_list = list(temp[:, 0])    # 取出第0列数据，即图片路径

    all_label_list = list(temp[:, 1])    # 取出第1列数据，即图片标签
#     print(all_image_list,'******', all_label_list)
    
    label_list = [int(i) for i in label_list]   # 转换成int数据类型


    ''' 

    # 将所得List分为两部分，一部分用来训练tra，一部分用来测试val

    n_sample = len(all_label_list)

    n_val = int(math.ceil(n_sample * ratio))  # 测试样本数, ratio是测试集的比例

    n_train = n_sample - n_val  # 训练样本数



    tra_images = all_image_list[0:n_train]

    tra_labels = all_label_list[0:n_train]

    tra_labels = [int(float(i)) for i in tra_labels]   # 转换成int数据类型

    val_images = all_image_list[n_train:-1]

    val_labels = all_label_list[n_train:-1]

    val_labels = [int(float(i)) for i in val_labels]   # 转换成int数据类型



    return tra_images, tra_labels, val_images, val_labels

    '''

    return image_list, label_list

# file_dir = 'G:/MNIST_data/test/'
# get_file(file_dir)

# 将image和label转为list格式数据，因为后边用到的的一些tensorflow函数接收的是list格式数据

# 为了方便网络的训练，输入数据进行batch处理

# image_W, image_H, ：图像高度和宽度

# batch_size：每个batch要放多少张图片

# capacity：一个队列最大多少

def get_batch(image, label, image_W, image_H, batch_size, capacity):

    # step1：将上面生成的List传入get_batch() ，转换类型，产生一个输入队列queue

    # tf.cast()用来做类型转换

    image = tf.cast(image, tf.string)   # 可变长度的字节数组.每一个张量元素都是一个字节数组

    label = tf.cast(label, tf.int32)

    # tf.train.slice_input_producer是一个tensor生成器

    # 作用是按照设定，每次从一个tensor列表中按顺序或者随机抽取出一个tensor放入文件名队列。

    input_queue = tf.train.slice_input_producer([image, label])

    label = input_queue[1]

    image_contents = tf.read_file(input_queue[0])   # tf.read_file()从队列中读取图像

 

    # step2：将图像解码，使用相同类型的图像

    image = tf.image.decode_jpeg(image_contents, channels=3)

    # jpeg或者jpg格式都用decode_jpeg函数，其他格式可以去查看官方文档

 

    # step3：数据预处理，对图像进行旋转、缩放、裁剪、归一化等操作，让计算出的模型更健壮。

    image = tf.image.resize_image_with_crop_or_pad(image, image_W, image_H)

    # 对resize后的图片进行标准化处理

    image = tf.image.per_image_standardization(image)

 

    # step4：生成batch

    # image_batch: 4D tensor [batch_size, width, height, 3], dtype = tf.float32

    # label_batch: 1D tensor [batch_size], dtype = tf.int32

    image_batch, label_batch = tf.train.batch([image, label], batch_size=batch_size, num_threads=16, capacity=capacity)

 

    # 重新排列label，行数为[batch_size]

    label_batch = tf.reshape(label_batch, [batch_size])

    # image_batch = tf.cast(image_batch, tf.uint8)    # 显示彩色图像

    image_batch = tf.cast(image_batch, tf.float32)    # 显示灰度图

    # print(label_batch) Tensor("Reshape:0", shape=(6,), dtype=int32)

    return image_batch, label_batch

    # 获取两个batch，两个batch即为传入神经网络的数据
# get_batch(image, label, image_W, image_H, batch_size, capacity)
 



def PreWork():

    # 对预处理的数据进行可视化，查看预处理的效果

    IMG_W = 28

    IMG_H = 28

    BATCH_SIZE = 6

    CAPACITY = 64



    train_dir = 'G:/MNIST_data/test/'



    # image_list, label_list, val_images, val_labels = get_file(train_dir)

    image_list, label_list = get_file(train_dir)

    image_batch, label_batch = get_batch(image_list, label_list, IMG_W, IMG_H, BATCH_SIZE, CAPACITY)

    print(label_batch.shape)



    lists = ('0', '1', '2', '3', '4', '5')



    with tf.Session() as sess:

        i = 0

        coord = tf.train.Coordinator()  # 创建一个线程协调器，用来管理之后在Session中启动的所有线程

        threads = tf.train.start_queue_runners(coord=coord)

        try:

            while not coord.should_stop() and i < 1:

                # 提取出两个batch的图片并可视化。

                img, label = sess.run([image_batch, label_batch])  # 在会话中取出img和label

                # img = tf.cast(img, tf.uint8)



                '''

                1、range()返回的是range object，而np.arange()返回的是numpy.ndarray()

                range(start, end, step)，返回一个list对象，起始值为start，终止值为end，但不含终止值，步长为step。只能创建int型list。

                arange(start, end, step)，与range()类似，但是返回一个array对象。需要引入import numpy as np，并且arange可以使用float型数据。

                

                2、range()不支持步长为小数，np.arange()支持步长为小数

                

                3、两者都可用于迭代

                range尽可用于迭代，而np.nrange作用远不止于此，它是一个序列，可被当做向量使用。

                '''

                for j in np.arange(BATCH_SIZE):

                    # np.arange()函数返回一个有终点和起点的固定步长的排列

                    print('label: %d' % label[j])

                    plt.imshow(img[j, :, :, :])

                    title = lists[int(label[j])]

                    plt.title(title)
                    
                    plt.axis('off')
                    
                    plt.show()

                i += 1

        except tf.errors.OutOfRangeError:

            print('done!')

        finally:

            coord.request_stop()

        coord.join(threads)



if __name__ == '__main__':

    PreWork()


